In [1]:
import sys
sys.path.append("../../helios-server/")
import helios.models as models
from helios_auth import models as auth_model
import helios.views as views
from helios.models import CastVote, Voter, Trustee
from helios.workflows.homomorphic import EncryptedVote, Tally, EncryptedAnswer
from helios.crypto import algs
from election_utils import *
from tqdm import tqdm, trange
# import click
import time
import os
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
import pickle

In [3]:
# num_questions = 4
num_choices = 4
choices = [i for i in range(num_choices)]

In [4]:
possible_answers = []
for i in trange(num_choices+1):
    possible_answers.extend([list(xs) for xs in combinations(choices, i)])

100%|██████████| 5/5 [00:00<00:00, 6773.75it/s]


In [5]:
possible_answers

[[],
 [0],
 [1],
 [2],
 [3],
 [0, 1],
 [0, 2],
 [0, 3],
 [1, 2],
 [1, 3],
 [2, 3],
 [0, 1, 2],
 [0, 1, 3],
 [0, 2, 3],
 [1, 2, 3],
 [0, 1, 2, 3]]

In [6]:
election_keypair = pickle.load(open("election_keypair.p", "rb"))

In [40]:
def fromElectionAndAnswer(choice_indicies, answer, pk, q_max, q_min=0):
    """
    Given an election, a question number, and a list of answers to that question
    in the form of an array of 0-based indexes into the answer array,
    produce an EncryptedAnswer that works.
    """
    
    # initialize choices, individual proofs, randomness and overall proof
    choices = [None for a in range(len(choice_indicies))]
    individual_proofs = [None for a in range(len(choice_indicies))]
    overall_proof = None
    randomness = [None for a in range(len(choice_indicies))]
    
    # possible plaintexts [0, 1]
    plaintexts = EncryptedAnswer.generate_plaintexts(pk)
    
    # keep track of number of options selected.
    num_selected_answers = 0
    
    # homomorphic sum of all
    homomorphic_sum = 0
    randomness_sum = 0

    # min and max for number of answers, useful later

    min_answers = q_min
    max_answers = q_max

    # go through each possible answer and encrypt either a g^0 or a g^1.
    for answer_num in range(len(choice_indicies)):
      plaintext_index = 0
      
      # assuming a list of answers
      if answer_num in answer:
        plaintext_index = 1
        num_selected_answers += 1

      # randomness and encryption
      randomness[answer_num] = algs.random.mpz_lt(pk.q)
      choices[answer_num] = pk.encrypt_with_r(plaintexts[plaintext_index], randomness[answer_num])
      
      # generate proof
      individual_proofs[answer_num] = choices[answer_num].generate_disjunctive_encryption_proof(plaintexts, plaintext_index, 
                                                randomness[answer_num], algs.EG_disjunctive_challenge_generator)
                                                
      # sum things up homomorphically if needed
      if max_answers is not None:
        homomorphic_sum = choices[answer_num] * homomorphic_sum
        randomness_sum = (randomness_sum + randomness[answer_num]) % pk.q

    # prove that the sum is 0 or 1 (can be "blank vote" for this answer)
    # num_selected_answers is 0 or 1, which is the index into the plaintext that is actually encoded
    
    if num_selected_answers < min_answers:
      raise Exception("Need to select at least %s answer(s)" % min_answers)
    
    if max_answers is not None:
      sum_plaintexts = EncryptedAnswer.generate_plaintexts(pk, min=min_answers, max=max_answers)
    
      # need to subtract the min from the offset
      overall_proof = homomorphic_sum.generate_disjunctive_encryption_proof(sum_plaintexts, num_selected_answers - min_answers, randomness_sum, algs.EG_disjunctive_challenge_generator);
    else:
      # approval voting
      overall_proof = None
    
    return EncryptedAnswer(choices, individual_proofs, overall_proof, randomness, answer)

In [41]:
for num_choices in range(1, 2):
    choices = [i for i in range(num_choices)]
    print(f"Enumerating possible answers with {num_choices} choices")
    possible_answers = []
    for i in trange(num_choices+1):
        possible_answers.extend([list(xs) for xs in combinations(choices, i)])
    
    print(f"Generating encyptions")
    encypted_answers = [fromElectionAndAnswer(choices, answer, election_keypair["public_key"], num_choices) for answer in tqdm(possible_answers)]
    # np.save(f"encrypted_answers/all_answers_with_{num_choices}_choices.npy", encypted_answers,  allow_pickle=True)

Enumerating possible answers with 1 choices



100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 20213.51it/s]


Generating encyptions



100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 14.03it/s]


In [49]:
encypted_answers[0].choices[0].decrypt(decryption_factors, pk)

TypeError: unsupported operand type(s) for //: 'list' and 'int'

In [27]:
list(range(1, 5))

[1, 2, 3, 4]

In [56]:
xs = np.load(f"encrypted_answers/all_answers_with_4_choices.npy",  allow_pickle=True)
election_keypair = pickle.load(open("election_keypair.p", "rb"))
decryption_factors = election_keypair["decryption_factors"]
pk = election_keypair["public_key"]

q_num = 0
a_num = 0
dec_factor_list = [df[q_num][a_num] for df in decryption_factors]

In [57]:
xs[0].choices[0].decrypt(dec_factor_list, pk)

14678644347333918513058728299899450179582154072056843026102007728222895313808169161575312889897792251157871599030510103870225831966092631074349926929537163597921628815127881669625144857215285256367771341227890184649381005214179606883402346839994515823244730172970724637737142889871850814078098128705542748597020063352474692844791965309609425996757664699877385247077645169080192197328706239577305996168697465438050189872574048388246584589859151443202783188560123437563106355765312936877980794123864806609974606923193600267009752548803419834114991756961715520926756108099484391291103363672376618345778046692160576457615

In [58]:
pk.g

14887492224963187634282421537186040801304008017743492304481737382571933937568724473847106029915040150784031882206090286938661464458896494215273989547889201144857352611058572236578734319505128042602372864570426550855201448111746579871811249114781674309062693442442368697449970648232621880001709535143047913661432883287150003429802392229361583608686643243349727791976247247948618930423866180410558458272606627111270040091203073580238905303994472202930783207472394578498507764703191288249547659899997131166130259700604433891232298182348403175947450284433411265966789131024573629546048637848902243503970966798589660808533

In [18]:
xs[1].choices[0].__dict__

{'pk': <helios.crypto.elgamal.PublicKey at 0x113076610>,
 'alpha': 7624700057623326213293990262890455027670034793713012268645240409248405844116365675186901125389192668950808585248302931164406352548096341348313734139444049975166675941100538515919441406760785565678840758682301824355543044133434624979503178268786168743061154410113276484409524239477825231412639148058495290234484078672549994988383256682230859587302830162177207606782300761563742524786309091523523595723084380519686989933049693404818872872400671914615065229129236058683459205188324821889415651438184837950382883008253727663610854644573354995083747952883950995761342030117357917378497632279112277689324744406606392295730,
 'beta': 108140544620183266469409658798322210925296881589820201177770313304412000714791549162875295128415659911787405582273747462245773279952035869680202009921540342075541475554811002108766933666371908019297916784187828049042394391526196811851630054737609118285646118518072500332922171737039140165714169237892439937

In [59]:
pk.q

61329566248342901292543872769978950870633559608669337131139375508370458778917